In [1]:
import numpy as np
from boxvectors import directions as directions
import Initial_Parameters as ip
from md import System
from md import md
from distribution import maxwellboltzmann
import matplotlib.pyplot as plt
from scipy.special import erf
from scipy.special import erfc
from scipy.constants import epsilon_0
%matplotlib inline

In [2]:
Symbols = ip.Symbols
Coefficients = ip.Coefficients
Charges = ip.Charges
N = ip.N*np.sum(Coefficients)
L = ip.L
T = ip.T
dt = ip.dt
p_rea = ip.p_rea
std = ip.std
n_boxes_short_range = ip.n_boxes_short_range
p_error = ip.p_error
Sys= System(Symbols, Coefficients, Charges, N/2)
Labels = Sys.get_Labels()
Sigma, Epsilon = Sys.get_LJ_parameter()
r_cut_coulomb = ip.r_cut_coulomb
r_cut_LJ = ip.r_cut_LJ
r_switch = ip.r_switch
switch_parameter = ip.switch_parameter
m = Labels[:,0]

In [3]:
def get_random_starting_Positions(N,L):
    Positions = np.zeros((N,3))
    Positions[:,0] = np.linspace(0,L[0],N, endpoint = False)
    Positions[:,1] = np.linspace(0,L[1],N, endpoint = False)
    Positions[:,2] = np.linspace(0,L[2],N, endpoint = False)
    np.random.shuffle(Positions[:,0])
    np.random.shuffle(Positions[:,1])
    np.random.shuffle(Positions[:,2])
    return Positions
Positions = get_random_starting_Positions(N,L)
Velocities = maxwellboltzmann().sample_distribution(N,m,T)
Forces = np.zeros((N,3))
# R = np.linalg.norm(Positions,axis=1)

In [4]:
# Positions = ((directions(2).get_directions()))
# Positions = Positions +2
# Positions = np.delete(Positions, np.where(np.max(Positions, axis = 1)==4),axis = 0)
# Positions = (Positions*564e-12)+282e-12
R = np.linalg.norm(Positions,axis=1)

In [5]:
L*=1.5

In [6]:
MD = md(
    Positions, 
    Labels, 
    Velocities,
    Forces, 
    L, 
    T, 
    Sigma, 
    Epsilon, 
    switch_parameter,
    r_switch,
    r_cut_LJ,
    dt, 
    std,
    n_boxes_short_range,
    p_rea,
    p_error,
    Symbols)

0.0160000324249


neighbourlist.py:44: RuntimeWarning: divide by zero encountered in double_scalars
  r_c = box_length / n_cells


In [7]:
MD.forces = MD.get_forces()

In [8]:
# print MD.get_energy()
# print MD.get_potential()
# print MD.forces

In [9]:
import os
import time
cwd = os.getcwd()
cwd

'C:\\Users\\Marco\\Documents\\GitHub\\compsci-proj-md-master\\md'

In [10]:
start = time.time()
MD.minmimize_Energy(N_steps = 1000, threshold=1e-11, Energy_save=1,  Frame_save=1, constant = dt, path=cwd)
ende = time.time()
print "\n"
print ende-start

100.0% of steps completed

131.890000105


In [11]:
start = time.time()
MD.get_traj(N_steps=100, Energy_save=1, Temperature_save=1, Frame_save=1, path = cwd)
ende = time.time()
print "\n"
print ende-start

100.0% of steps \completed

13.1859998703


In [12]:
MD.forces

array([[ -4.01394385e-11,   3.15363715e-11,   2.76309015e-11],
       [  4.11849532e-12,   1.70987356e-11,  -3.22270284e-11],
       [ -3.19392451e-11,  -3.55588586e-11,  -9.59827024e-12],
       [ -6.98429091e-12,   4.14449537e-11,   2.27361340e-11],
       [  3.54497360e-11,  -4.80762551e-11,   3.90466657e-11],
       [ -3.47992187e-11,  -5.03366123e-11,   1.11193502e-11],
       [  2.50093051e-12,   4.05080015e-11,  -1.56713054e-11],
       [ -4.34619026e-11,   5.18913698e-11,   5.09108812e-11],
       [ -4.18995125e-11,  -4.37634357e-11,  -1.04515093e-12],
       [  1.04151222e-11,   6.84264675e-11,  -3.88458795e-12],
       [  2.60483657e+03,  -2.41568307e+03,   1.00885026e+03],
       [ -1.22526568e-11,   3.47824440e-11,  -2.37651485e-11],
       [ -9.94318213e-14,   6.76974711e-11,  -3.61318615e-12],
       [ -1.41112537e-11,   4.01251524e-11,  -5.48562840e-11],
       [  9.18008909e-12,   3.06882046e-11,  -5.09523470e-11],
       [ -1.37877562e-12,   6.40309843e-11,   2.0354643